In [ ]:
import warnings
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('/content/Drive/MyDrive/475/train.csv')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df[' Source IP'])
df.drop(" Source IP", axis=1, inplace=True)

# Appending the array to our dataFrame
# with column name 'Purchased'
df[" Source IP"] = label
 le = LabelEncoder()

# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df[' Timestamp'])
df.drop(" Timestamp", axis=1, inplace=True)

# Appending the array to our dataFrame
# with column name 'Purchased'
df[" Timestamp"] = label
le = LabelEncoder()

# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df[' Destination IP'])
df.drop(" Destination IP", axis=1, inplace=True)

# Appending the array to our dataFrame
# with column name 'Purchased'
df[" Destination IP"] = label


In [ ]:
#Clustering
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data


X = df
X, labels_true = make_blobs(
    n_samples=750, cluster_std=0.4, random_state=0
)


# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
home=metrics.homogeneity_score(labels_true, labels)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
print("Homogeneity: %0.3f" % home)
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels_true, labels))
print(
    "Adjusted Mutual Information: %0.3f"
    % metrics.adjusted_mutual_info_score(labels_true, labels)
)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=14,
    )

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=6,
    )

plt.title("Estimated number of clusters: %d" % n_clusters_)
plt.show()


In [ ]:
#ARIMA
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
#from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
# load dataset
#def parser(x):
#	return datetime.strptime('190'+x, '%Y-%m')
#series = read_csv('shampoo-sales.csv')
#series.index = series.index.to_period('M')
 #split into train and test sets
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
# walk-forward validation
for t in range(len(test)):
	model = ARIMA(history, order=(1,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
#Lypunav
from math import log1p
import numpy as np
import matplotlib.pyplot as plt
import time
t1 = time.perf_counter()
with open('/content/Drive/MyDrive/timeseries.txt', 'r') as f:
    data = [float(i) for i in f.read().split()]

N = len(data)
eps = 0.001
lyapunovs = [[] for i in range(N)]

for i in range(N):
    for j in range(i + 1, N):
        if np.abs(data[i] - data[j]) < eps:
            for k in range(min(N - i, N - j)):
                lyapunovs[k].append(log1p(np.abs(data[i+k] - data[j+k])))

with open('lyapunov.txt', 'w') as f:
    for i in range(len(lyapunovs)):
        if len(lyapunovs[i]):
            string = str(np.round( sum(lyapunovs[i]) / len(lyapunovs[i])))
            f.write(string + '\n')
t2 = time.perf_counter()
t3=t2-t1
print('time taken to run in sec:',t3)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import boxcox
from nolds import lyap_e

# Predict values
predictions
# Calculate prediction errors
prediction_errors = np.abs(Z_test - predictions)

# Calculate Lyapunov exponent
lyapunov_exponent = lyap_e(prediction_errors, emb_dim=10, min_tsep=10, min_neighbors=20)

# Determine anomaly scores
anomaly_scores = np.where(lyapunov_exponent > threshold_lambda, 1, 0)

# Plot the Lyapunov exponent values
plt.figure(figsize=(12, 6))
plt.plot(lyapunov_exponent, label="Lyapunov Exponent")
plt.axhline(y=threshold_lambda, color='r', linestyle='--', label="Threshold")
plt.xlabel("Time")
plt.ylabel("Lyapunov Exponent")
plt.legend()
plt.grid(True)
plt.show()

# Print or analyze the anomaly scores
print("Anomaly Scores:")
print(anomaly_scores)


In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import medfilt

# Load or generate your data as a Pandas DataFrame, replace 'your_data.csv' with your dataset
data = pd.read_csv('/content/Drive/MyDrive/475/train.csv')

# Define constants
alpha1 = 0.1
alpha2 = 0.8
q = 3  # Adjust this multiplier as needed for thresholding

# Function to calculate the absolute difference between two exponential filters
def absolute_difference(x, alpha1, alpha2):
    f1 = f2 = 0
    adf = []

    for val in x:
        f1 = alpha1 * val + (1 - alpha1) * f1
        f2 = alpha2 * val + (1 - alpha2) * f2
        adf.append(abs(f1 - f2))

    return adf

# Function to calculate the rolling median
def rolling_median(x, window_size):
    return medfilt(x, kernel_size=window_size)

# Function to calculate least distance and threshold for anomaly detection
def anomaly_detection(x, ld_mean, ld_std, q):
    ldt = abs(x - ld_mean)
    threshold = ld_mean + q * ld_std
    score2 = 0 if ldt <= threshold else 1
    return score2

# Process the NUDA feature
NUDA_feature = data['Destination IP'].values
Adf = absolute_difference(NUDA_feature, alpha1, alpha2)
M = rolling_median(Adf, window_size=5)  # Adjust the window size as needed

# Compute the mean and standard deviation of least distances
ld = []
for i in NUDA_feature:
    ld.append(min(abs(i - x) for x in NUDA_feature if x != i))
ld_mean = np.mean(ld)
ld_std = np.std(ld)

# Set Model 2 to False as described in the algorithm
Model_2 = False

# Loop through incoming traffic instances
testdata= data[ Destination IP]
for yt in testdata  # Replace 'incoming_traffic' with your incoming data
    ldt = abs(yt - NUDA_feature)
    score2 = anomaly_detection(ldt, ld_mean, ld_std, q)

    # Check if it's abnormal and take appropriate action
    if score2 == 1:
        # Implement your action for handling abnormal traffic here
        print("Abnormal Traffic Detected")
    else:
        print("Normal Traffic")


In [ ]:
# Initialize variables
threshold = 0.6  # Adjust threshold as needed

# Define a function for event correlation
def event_correlation(cluster_scores):
    # Calculate the sum of anomaly scores for all clusters using AND operation
    final_scores = [score1 and score2 for score1, score2 in cluster_scores]

    # Check if the sum of final anomaly scores is greater than the threshold
    total_score = sum(final_scores)
    if total_score > threshold:
        return "Abnormal Traffic"
    else:
        return "Normal Traffic"

# Main loop to process incoming traffic samples
while True:
    incoming_traffic = receive_traffic_sample()  # Receive a new traffic sample

    # Process the traffic sample and obtain anomaly scores for each cluster
    cluster_scores = []
    for cluster in clusters:
        score1, score2 = calculate_anomaly_scores(incoming_traffic, cluster)
        cluster_scores.append((score1, score2))

    # Perform event correlation to determine traffic type
    traffic_type = event_correlation(cluster_scores)


    if traffic_type == "Abnormal Traffic":

        print("Abnormal Traffic Detected")
